<!--NAVIGATION-->

<a href="https://colab.research.google.com/github/bpesquet/machine-learning-katas/blob/master/classic-datasets/Iris.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Guided ML With The Iris Dataset

| Learning type | Activity type | Objective |
| - | - | - |
| Supervised | Multiclass classification | Identify a flower's class |

Contents:
1. Loading the data
2. Setting up supervised learning problem (selecting features)
3. Creating a first model
    - Creating train and test datasets
    - Normalizing train and test
    - Fitting and predicting
4. Evaluate the frist model predictions
5. Crossvalidation of the model
6. Creating an end to end ML pipeline
    - Train/Test Split
    - Normalize
    - Crossvalidations
    - Model
    - fitting and predicting

## Instructions with NBGrader removed

Complete the cells beginning with `# YOUR CODE HERE` and run the subsequent cells to check your code.

## About the dataset

[Iris](https://archive.ics.uci.edu/ml/datasets/iris) is a well-known multiclass dataset. It contains 3 classes of flowers with 50 examples each. There are a total of 4 features for each flower.

![](./classic-datasets/images/Iris-versicolor-21_1.jpg)

## Package setups

1. Run the following two cells to initalize the required libraries. 

In [1]:
#to debug package errors
import sys
sys.path
sys.executable

'/Users/ns/anaconda3/envs/bk_py3/bin/python'

In [1]:
# Import needed packages
# You may add or remove packages should you need them
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import make_pipeline

# Set random seed
np.random.seed(0)

# Display plots inline and change plot resolution to retina
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# Set Seaborn aesthetic parameters to defaults
sns.set()

## Step 1: Loading the data

1. Load the iris dataset using ```datasets.load_iris()```
2. Investigate the data structure with ```.keys()```
3. Construct a dataframe from the dataset
4. Create a 'target' and a 'class' column that contains the target names and values
5. Display a random sample of the dataframe 

In [2]:
#Your code here.
data_iris = datasets.load_iris()
print(f"data.keys() :{data_iris.keys()}")
df = pd.DataFrame(data_iris.data)
df.columns = data_iris.feature_names
df['target'] = data_iris.target
df['class'] = df['target'].apply(lambda x: data_iris['target_names'][x])

data.keys() :dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])


In [3]:
df.sample(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,class
114,5.8,2.8,5.1,2.4,2,virginica
62,6.0,2.2,4.0,1.0,1,versicolor
33,5.5,4.2,1.4,0.2,0,setosa
107,7.3,2.9,6.3,1.8,2,virginica
7,5.0,3.4,1.5,0.2,0,setosa
100,6.3,3.3,6.0,2.5,2,virginica
40,5.0,3.5,1.3,0.3,0,setosa
86,6.7,3.1,4.7,1.5,1,versicolor
76,6.8,2.8,4.8,1.4,1,versicolor
71,6.1,2.8,4.0,1.3,1,versicolor


### Question
Find the X and y values we're looking for. Notice that y is categorical and thus, we could **one-hot encode it** if we are looking at **class** or we can just pick **target**. In order to one hot encode we have  to re-shape `y` it using the **.get_dummies** function. 

#### For the purpose of this exercise, do not use hot encoding, go only for target but think about if you have to drop it somewhere or not...

In [4]:
# YOUR CODE HERE
predictors = ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']
X = df[predictors]
y = df.target
y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int32

## Step 2: Setting up supervised learning problem (selecting features)

Feature selection is an essential step in improving a model's perfromance. In the first version of the model we will use the 'sepal length' and 'sepal width' as predicting features. Later we will see the effect of adding additional features.

1. Assign the values of the 'target' to Y as a numpy array
2. Assign the remaining feature values to X as a numpy array
3. Check the shape of X and Y. Check the first few values.
    - Can we confirm our X and Y are created correctly?

In [5]:
#your code here

X = df[['sepal length (cm)','sepal width (cm)']].values  # .reshape(-1,2)
Y = df['target'].values
print(Y.shape)
print(Y[0:5])
print(X.shape)
print(X[0:5])

(150,)
[0 0 0 0 0]
(150, 2)
[[5.1 3.5]
 [4.9 3. ]
 [4.7 3.2]
 [4.6 3.1]
 [5.  3.6]]


## Step 3: Creating the first model

In lecture we learned about creating a train and test datasets, normalizing, and fitting a model. In this step we will see how to build a simple version of this.

We have to be careful when constructing our train and test datasets. First, when we create train and test datasets we have to be careful that we always have the same datapoints in each set. Otherwise our results won't be reproduceable or we might introduce a bias into our model.

We also need to be attentive to when we normalize the data. What would be the effect of normalizing the data (i.e. with StandardScaler to a range between 0 - 1) before we create our train and test sets? Effectively we would use information in the test set to structure the values in the training set and vice versa. Therefore normalizing train and test independently is the preferred method.

1. Create X_train, X_test, Y_train, Y_test using ```train_test_split()``` with an 80/20 train/test split. Look in the SKLearn documentation to understand how the function works.
    - Inspect the first few rows of X_train.
    - Run the cell a few times. Do the first few rows change?
    - What option can we use in ```train_test_split()``` to stop this from happening?
2. Normalize the train and test datasets with ```StandardScaler```
    - We can fit the transform with ```.fit()``` and ```.transform()``` to apply it. Look in the documentation for an esample of how to do this.
    - Does it make sense to normalize Y_train and Y_test?
3. Initalize a ```LogisticRegression()``` model and use the ```.fit()``` method to initalize the first model.
    - We will pass the X_train and Y_train variables to the ```.fit()``` method.
    - Once the model is fit, use the ```.predict()``` with the X_test and save the output as predictions.

In [6]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X_scaled = scalar.fit_transform(X)

In [7]:
#split train and test data 80/20
#your code here
X_train, X_test, y_train, y_test = train_test_split(X_scaled,Y, test_size=0.20, random_state=1)
X_train[0:5]

array([[ 0.31099753, -0.13197948],
       [ 2.24968346, -0.13197948],
       [-0.29484182, -1.28296331],
       [ 0.67450115, -0.59237301],
       [-0.05250608, -0.59237301]])

initalize and fit with an [SVC Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC), note that this is **different than in the previous exercise!**




In [13]:
#your code here
from sklearn.svm import LinearSVC, SVC
svc_clf = SVC(kernel='linear')
svc_model = svc_clf.fit(X_train,y_train)
y_pred_svc = svc_model.predict(X_test)

## Step 4: Evaluate the first model's predictions

We will learn more about how to evaluate the performance of a classifier in later lessons. For now we will use % accuracy as our metric. It is important to know that this metric only helps us understand the specific performance of our model and not, for example, where we can improve it, or where it already perfoms well.

1. Use ```.score()``` to evaluate the performance of our first model.

In [14]:
#evaluating the performace of our first model
#your code here
svc_train_score = svc_model.score(X_train,y_train)
print(f"Accuracy for TRAIN set: {svc_train_score}")

svc_test_score = svc_model.score(X_test,y_test)
print(f"Accuracy for TEST set: {svc_test_score}")


Accuracy for TRAIN set: 0.8083333333333333
Accuracy for TEST set: 0.8


## Step 5: Crossvalidation of the model
Our first model achived:  **Insert your % accuracy**!
- How do we know it is reproducible? 
- If we run the model again and our performance is 85%, which is correct? 
- And what about improving our model? Can you think of one thing we can do to potentially improve the model?

#### Crossvalidation
[Crossvalidation](https://machinelearningmastery.com/k-fold-cross-validation/) is when we create multiple X and y datasets. On each dataset we train and fit the model. We then average the results and return a 'crossvalidated' accuracy.

1. Initalize a new version of the model you trained above with the same paramters.
2. Use ```cross_validate()``` to run the model with 5 crossvalidation folds. 

In [17]:
from sklearn.model_selection import cross_val_score, cross_validate
cv = cross_validate(svc_model, X_scaled, y, cv=5)
i = 0
for acc in cv['test_score']:
    print(f'Crossvalidation fold:{i}  Accruacy: {acc}')
    i += 1
mean = cv['test_score'].mean()
print(f'\nMean train cross validation score {mean}')

Crossvalidation fold:0  Accruacy: 0.7333333333333333
Crossvalidation fold:1  Accruacy: 0.8333333333333334
Crossvalidation fold:2  Accruacy: 0.7666666666666667
Crossvalidation fold:3  Accruacy: 0.8666666666666667
Crossvalidation fold:4  Accruacy: 0.8666666666666667

Mean train cross validation score 0.8133333333333332


## Step 6: Creating an end to end ML pipeline
**Congraulations you've trained, crossvalidated, predicted, and evaluated your first classifier**. Now that you understand the basic steps we will look at a way to combine all these steps together.

Before we go further think about what you would have to do if you **wanted to change the model**. Intalize a new model, change the variables, redo the cross validation...etc. Seems like a lot. And when we have to **change lots of code** it is **easy to make mistakes**. 
- And what if you wanted to try many models and see which one performed best? 
- Or try changing many different features? 
- How could you do it without writing each one out as we have?

The solution is to use **SKLearn's pipeline class**. A pipeline is an object that will execute the various steps in the machine learning process. We can choose what elements we want in the pipeline and those that we do not. Once setup, we can rapidly change models, or input data and have it return our results in an ordered way. Check out this [tutorial](https://medium.com/vickdata/a-simple-guide-to-scikit-learn-pipelines-4ac0d974bdcf)


1. Initalize a scaler and a classifer object like we did previously.
2. Use the ```make_pipeline()``` function to construct a transofmraiton pipeline for the scaler and the classifier
3. Input the pipeline object to the cross_validator and evaluate with 5 folds.
4. Print out your results (hint: make a function for repetitve tasks like printing)

In [22]:
predictors = ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']
X = df[predictors]
Y = df.target
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.20, random_state=1)

from sklearn.pipeline import make_pipeline
svc_pipeline = make_pipeline(StandardScaler(), SVC(kernel='linear'))

svc_pipeline.fit(X_train,y_train)
y_pred_svc = svc_pipeline.predict(X_test)

In [23]:
from sklearn.model_selection import cross_val_score, cross_validate
cv = cross_validate(svc_pipeline, X, Y, cv=5)
i = 0
for acc in cv['test_score']:
    print(f'Crossvalidation fold:{i}  Accruacy: {acc}')
    i += 1
mean = cv['test_score'].mean()
print(f'\nMean train cross validation score {mean}')

Crossvalidation fold:0  Accruacy: 0.9666666666666667
Crossvalidation fold:1  Accruacy: 1.0
Crossvalidation fold:2  Accruacy: 0.9333333333333333
Crossvalidation fold:3  Accruacy: 0.9333333333333333
Crossvalidation fold:4  Accruacy: 1.0

Mean train cross validation score 0.9666666666666668


## Even more challenging Exercise

In this notebook we only used two features to predict the class of the flower. We also did not do any hypter parameter tuning. The challenge is to impove the prediction results. Some ideas we can try:
1. Add features to the input and run the cross validation pipeline
2. Investigate how to use ```GridSearchCV```, a powerful funtion that searches through hyperparmetrs and does cross validation.
    - Hint: Input the pipeline directly into GridSearchCV
3. Try a different models like RandomForest or SVM.

In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_svc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      0.92      0.96        13
           2       0.86      1.00      0.92         6

    accuracy                           0.97        30
   macro avg       0.95      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30



In [29]:
#your challenge code here
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

c = np.logspace(start = 0, stop = 10, num = 100, base = 2 , dtype = 'float64')
param_grid = {'C': c}

grid = GridSearchCV(svc_clf, param_grid =param_grid, cv=10, n_jobs=-1, scoring='accuracy')
grid.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.0f" % (grid.best_params_, grid.best_score_ * 100 ))
print( "Best estimator accuracy on test set {:.2f} ".format(grid.best_estimator_.score(X_test, y_test) * 100 ))

The best parameters are {'C': 1.0} with a score of 98
Best estimator accuracy on test set 100.00 


In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_svc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      0.92      0.96        13
           2       0.86      1.00      0.92         6

    accuracy                           0.97        30
   macro avg       0.95      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30

